In [62]:
#notebook for TD HUU randomization report

In [63]:
# Imports
import os 
import pandas as pd
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date
import csv
from django_pandas.io import read_frame

In [64]:
consents_pids = SubjectConsent.objects.values_list('screening_identifier', flat=True).distinct()


#Consented HUU
consented_md = MaternalDataset.objects.filter(protocol='Tshilo Dikotla',
                                              mom_hivstatus='HIV-uninfected',
                                              screening_identifier__in=consents_pids)

consented_md_pids = consented_md.values_list('study_maternal_identifier', flat=True)

#Unconsented HUU

td_uncosent_md = MaternalDataset.objects.filter(protocol='Tshilo Dikotla',
                                      mom_hivstatus='HIV-uninfected').exclude(
    screening_identifier__in=consents_pids)

print("Total Consented Caregivers: ", len(consents_pids))
print("Total TD Caregivers: ", td_md.count())
print("Total TD HUU Caregivers: ", len(td_huu_md_pids))
print("Total TD Consented HUU: ", len(consented_md_pids))
print("Total TD Not Consented HUU: ", len(td_uncosent_pids))
print("Total TD Offstudy: ", len(offstudy))


data = [
    ['total_td_huu.csv', td_huu_md_pids],
    ['total_consented_td_huu.csv',  consented_md_pids],
    ['total_not_consented_td_huu.csv', td_uncosent_pids],
    ['total_offstudy_td_huu.csv', offstudy]
]

for d in data:
    with open(d[0], 'w') as f:
        writer = csv.writer(f)
        for val in d[1]:
            writer.writerow([val])

Total Consented Caregivers:  1053
Total TD Caregivers:  388
Total TD HUU Caregivers:  114
Total TD Consented HUU:  47
Total TD Not Consented HUU:  67
Total TD Offstudy:  0


In [65]:
#not_randomized
non_rando = WorkList.objects.filter(study_maternal_identifier__in=td_uncosent_md_pids, assigned=None)

# WorkList.objects.first().__dict__
print(non_rando.count())

Total in the worklist:  114
Total randomised TD HUU:  72
Total Not Randomised TD HUU:  0
Total missing from worklist:  0


In [5]:
#unconsented_HUU call logs
td_unconsent_log_entries = LogEntry.objects.filter(study_maternal_identifier__in=td_uncosent_md_pids)
td_unconsent_log_entries.count()

data_rando_dict = [
    worklist,randomised_worklist, final_not_randomised_worklist, missing_worklist_identifiers]

print("Total in the worklist: ", len(worklist))
print("Total randomised TD HUU: ", len(randomised_worklist))
print("Total Not Randomised TD HUU: ", len(final_not_randomised_worklist))
print("Total missing from worklist: ", len(missing_worklist_identifiers))

# importing library
import csv
 
    
data = [
    ['total_worklist_td_huu.csv', worklist],
    ['total_randomised_td_huu.csv', randomised_worklist],
    ['total_not_randomised_td_huu.csv', final_not_randomised_worklist],
    ['total_missing_worklist.csv', missing_worklist_identifiers]
]

for d in data:
    with open(d[0], 'w') as f:
        writer = csv.writer(f)
        for val in d[1]:
            writer.writerow([val])

In [66]:
unconsent_log_entry_ids = list(set(td_unconsent_log_entries.values_list('study_maternal_identifier', flat=True)))


#success contact unconsented huu
success_log_entries = LogEntry.objects.filter(~Q(phone_num_success=['none_of_the_above']),
                                             study_maternal_identifier__in=td_uncosent_md_pids)

success_log_entries_ids = success_log_entries.values_list('study_maternal_identifier', flat=True)

print("Attempts made: ", len(attempt_identifiers))
print('No Attempts: ', len(not_attempted))


data = [
    ['total_attempts_td_huu.csv', attempt_identifiers],
    ['total_notattempted_td_huu.csv',  not_attempted],
]

for d in data:
    with open(d[0], 'w') as f:
        writer = csv.writer(f)
        for val in d[1]:
            writer.writerow([val])

In [67]:
#Final decline
declined = LogEntry.objects.filter(study_maternal_identifier__in=success_log_entries_ids,
                                   appt='No')

declined_ids = declined.values_list('study_maternal_identifier', flat=True)


# All not continued contract and not consented

not_continued_contact_not_consented = list(set(td_uncosent_pids)-set(continued_contact))

qs = LogEntry.objects.filter(study_maternal_identifier__in=not_continued_contact_not_consented)

qs_data = []

for obj in qs:
    obj_dict = obj.__dict__
    del obj_dict['_state']
    del obj_dict['device_created']
    del obj_dict['device_modified']
    del obj_dict['modified']
    del obj_dict['created']
    del obj_dict['revision']    
    del obj_dict['user_created']
    del obj_dict['user_modified']
    del obj_dict['hostname_created']
    del obj_dict['hostname_modified']
    del obj_dict['log_id']
    
    qs_data.append(obj_dict)

df = pd.DataFrame(qs_data)
df.to_csv('log_entries_no_consent_not_continued_contact.csv', sep=',', encoding='utf-8')


data_attemps_dict = {'TD HUU Continued Contact': len(continued_contact)}

print('TD HUU Continued Contact: ', len(continued_contact))
print('TD HUU Not COntinued Contact Not Consented: ', len(not_continued_contact_not_consented))

data = [
    ['total_continued_contact_td_huu.csv', continued_contact],
    ['total_no_concent_not_continued_contact_td_huu.csv', not_continued_contact_not_consented]
]

for d in data:
    with open(d[0], 'w') as f:
        writer = csv.writer(f)
        for val in d[1]:
            writer.writerow([val])

TD HUU Continued Contact:  21
TD HUU Not COntinued Contact Not Consented:  46


In [68]:
may_not_call = LogEntry.objects.filter(study_maternal_identifier__in=success_log_entries_ids,
                                       may_call__icontains='no_')

may_not_call_ids = may_not_call.values_list('study_maternal_identifier', flat=True)

print('TD HUU Unreacheble: ', len(unreacheble))

data = [
    ['total_unreacheble_td_huu.csv', unreacheble]
]

for d in data:
    with open(d[0], 'w') as f:
        writer = csv.writer(f)
        for val in d[1]:
            writer.writerow([val])

TD HUU Unreacheble:  0


In [ ]:
#unsuccesssuccess contact unconsented huu
unsucessful_log_entries = LogEntry.objects.filter(phone_num_success=['none_of_the_above'],
                                                 study_maternal_identifier__in=td_uncosent_md_pids).exclude(
    study_maternal_identifier__in=success_log_entries_ids)

unsucessful_log_entries.count()

# Merge the 2 above lists
identifiers = screened_identifiers + declined_do_not_call
identifiers = list(set(identifiers) - set(consented_md_pids))

print('TD HUU Declined: ', len(identifiers))

data = [
    ['total_declined_td_huu.csv', identifiers]
]

for d in data:
    with open(d[0], 'w') as f:
        writer = csv.writer(f)
        for val in d[1]:
            writer.writerow([val])